In [1]:
pip install langchain langchain-groq sqlalchemy streamlit plotly pandas chroma pyngrok

In [2]:
from google.colab import userdata
import os
from langchain_groq import ChatGroq
import streamlit as st

try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY secret is empty or not found!")
    os.environ['GROQ_API_KEY'] = groq_api_key
except Exception as e:
    st.error(f"Failed to load Groq API key: {str(e)}\n\n"+
             "Fix: Left sidebar → key icon → Add secret 'GROQ_API_KEY' with your Groq key.")
    st.stop()

llm = ChatGroq(
    temperature=0.7,
    model_name="llama3-70b-8192")

try:
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    if not ngrok_token:
        raise ValueError("NGROK_AUTH_TOKEN secret is empty or not found!")
    os.environ['NGROK_AUTH_TOKEN'] = ngrok_token
except Exception as e:
    st.error(f"Failed to load NGROK_AUTH_TOKEN secret: {str(e)}\n\n"+
             "Fix: Left sidebar → key icon → Add secret 'NGROK_AUTH_TOKEN' with your ngrok auth token.")
    st.stop()

In [3]:
mkdir inventory-nl2sql-dashboard && cd inventory-nl2sql-dashboard

mkdir: cannot create directory ‘inventory-nl2sql-dashboard’: File exists


In [4]:
import sqlite3
import pandas as pd

db_path = "/content/inventory.db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
create table if not exists inventory (
  product_id integer primary key,
  name text not null,
  category text,
  quantity integer,
  price real,
  date DATE
)''')

In [5]:
sample_data = [
    (1, 'Laptop', 'Electronics', 50, 45000.0, '2023-01-15'),
    (2, 'Mouse', 'Electronics', 200, 500.0, '2023-02-20'),
    (3, 'Chair', 'Furniture', 100, 1500.0, '2023-03-10'),
    (4, 'Desk', 'Furniture', 30, 3000.0, '2023-04-05'),
    (5, 'Pen', 'Stationery', 500, 10.0, '2023-05-01'),
    (6, 'Notebook', 'Stationery', 150, 50.0, '2023-06-15'),
    (7, 'Monitor', 'Electronics', 20, 10000.0, '2023-07-20'),
    (8, 'Keyboard', 'Electronics', 80, 800.0, '2023-08-10'),
    (9, 'Table Lamp', 'Furniture', 60, 400.0, '2023-09-05'),
    (10, 'Stapler', 'Stationery', 300, 100.0, '2023-10-01')
]

In [6]:
cursor.executemany('insert or replace into inventory values (?,?,?,?,?,?)', sample_data)
conn.commit()
conn.close()
print(f"sqlite database created at db_path: {db_path}")

sqlite database created at db_path: /content/inventory.db


In [7]:
with open('/content/inventory.db', 'rb') as file:
  read_file = file.read()
  print(read_file)

b'SQLite format 3\x00\x10\x00\x01\x01\x00@  \x00\x00\x00\x04\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00.WJ\r\x00\x00\x00\x01\x0fM\x00\x0fM\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [8]:
import json

few_shot = [
    {"question" : "show me all electronics", "sql" : "select * from inventory where category = 'Electronics';"},
    {"question" : "what is total stock value", "sql" : " select sum(quantity * price) as total_value from inventory;"},
    {"question" : "low stock items under 500 rupees", "sql" : "select * from inventory where quantity < 50 and price < 500;"},
    {"question" : "items added after 2023-05-01", "sql" : "select * from inventory where date > '2023-05-01';"}]

with open('few_shot.json', 'w', encoding = 'utf-8') as f:
  json.dump(few_shot, f, indent = 2)

print("few_shot saved.")


few_shot saved.


In [32]:
# create the streamlit app file

%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import sqlite3
import json
import os

from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


# config
db_path = "/content/inventory.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

llm = ChatGroq(model = "llama-3.1-8b-instant",
               temperature = 0.2 )

# load few shot
with open("/content/few_shot.json", 'r') as f:
  few_shot_examples = json.load(f)

few_shot_prompt = "\n".join([f"user : {ex['question']}\n sql : {ex['sql']}" for ex in few_shot_examples])
def query_checker(q):
  q = q.replace("```sql", "").replace("```", "").strip()
  unsafe = ['delete', 'drop', 'update', 'insert']
  for word in unsafe:
    if word in q.lower():
      raise ValueError("unsafe query")
    return q

prompt = PromptTemplate.from_template(
"""
    you are a SQLite expert. generate only sql queries.
    never use delete, drop, uodate, insert.
    schema : {table_info}

    few_shot examples : {few_shot}

    question : {question}
    sql query :  """).partial(few_shot = few_shot_prompt)

chain = (
    RunnablePassthrough.assign(
        table_info=lambda _: db.get_table_info()
    )
    | prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(query_checker))


# streamlit ui
st.title("Inventory NL2SQL Dashboard (Colab)")

# simple query
query = st.text_input("ask in natural language", "show me low stock items in 1000 rupees")
if st.button("Run Query"):
  try:
    sql = chain.invoke({"question": query, "schema": db.get_table_info()})
    st.code(sql, language = "sql")

    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(sql, conn)
    conn.close()

    st.dataframe(df)
  except Exception as e:
    st.error(f"Error executing query: {e}")

# visuals (full_table)
st.subheader("Current Inventory")
conn = sqlite3.connect(db_path)
df_full = pd.read_sql_query("select * from inventory", conn)
conn.close()
st.dataframe(df_full)


fig = px.bar(df_full, x = 'name', y = 'quantity', color = 'category', title = 'stock by product')
st.plotly_chart(fig)

fig_pie = px.pie(df_full, names = 'category', values = 'quantity', title = 'category distribution')
st.plotly_chart(fig_pie)


Overwriting app.py


In [10]:
!pip install -U langchain-community langchain-core langchain-groq

In [33]:
from pyngrok import ngrok
import os
import time
from google.colab import userdata
import streamlit as st # Import streamlit for st.error and st.stop()

# Set ngrok authtoken
try:
  ngrok_api = userdata.get('NGROK_AUTH_TOKEN')
  if not ngrok_api:
      raise ValueError("NGROK_AUTH_TOKEN secret is empty or not found!")
  ngrok.set_auth_token(ngrok_api)
  print("ngrok token set successfully!")
except Exception as e:
  st.error(f"Failed to load NGROK_AUTH_TOKEN secret: {str(e)}\n\n"+
           "Fix: Left sidebar → key icon → Add secret 'NGROK_AUTH_TOKEN' with your ngrok auth token.")
  st.stop()

!pkill -f ngrok 2>/dev/null || true
ngrok.kill()

public_url = ngrok.connect(addr = '8501', bind_tls = True).public_url
print("your streamlit app should be live at ->", public_url)

!streamlit run app.py &>/dev/null&

ngrok token set successfully!
^C
your streamlit app should be live at -> https://uncaroled-paraesthetic-shirly.ngrok-free.dev


In [12]:
!ngrok authtoken 39vtKC2gzQMi7GZTObB7a5aJNFi_7BrntstoBJh1iwbnenSrE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
!pip install langchain-community langchain langchain-core langchain-groq sqlalchemy

In [15]:
!pip install -q --upgrade langchain langchain-community langchain-groq sqlalchemy streamlit plotly pandas pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [16]:

!pip install -q --upgrade \
    langchain \
    langchain-community \
    langchain-groq \
    sqlalchemy \
    streamlit \
    plotly \
    pandas \
    pyngrok

print("All packages installed/upgraded.")

All packages installed/upgraded.


In [17]:
# Test import — should print nothing (no error = success)
from langchain_community.utilities import SQLDatabase
print("langchain_community imported successfully!")

langchain_community imported successfully!


In [18]:
!pip install -q langchain-classic

In [19]:
pip show langchain

Name: langchain
Version: 1.2.10
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


In [20]:
!pip install langchain-community langchain-experimental langchain-openai

In [21]:
from langchain_openai import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities import SQLDatabase